In [1]:
from hdx.utilities.easy_logging import setup_logging
from hdx.hdx_configuration import Configuration
from hdx.data.dataset import Dataset
import pandas as pd
import ast
from scipy import spatial
from nltk.corpus import stopwords
import pandas as pd
from hdx.data.resource import Resource
import numpy as np
import nltk
import re 
import gensim
import os;
import json;
import re;
import logging;
import sqlite3;
import sys;
import multiprocessing;
import matplotlib.pyplot as plt;
from itertools import cycle;
from io import StringIO
import requests

import Utility

In [2]:
import warnings


warnings.filterwarnings('ignore')

## Crawling, Extracting, Collecting, and Preprocessing HDX datasets
<ol>
<li> Access HDX API and search for specific set of datasets.</li>
<li> Create a dataframe from the json of datasets. </li>
<li> Extract resource objects for each dataset (if the dataset has any). </li>
<li> Extract the header (column names) of the datsets if it has url (public url). </li>
<li> Save content to a file for further processing. </li>
<li> Eliminate stop words </li> 
</ol>


In [3]:
## add the 'geodata' to the dataset's metadata 

def add_geodata(df):
    for i, row in df.iterrows():
        if df.ix[i]['has_geodata'] == True:
            df.at[i,'geodata'] = "geodata"
    else:
            df.at[i,'geodata'] = " "
    return df

In [4]:
## add the country(ies) to the dataset's metadata

def add_country(df):
    for i, row in df.iterrows():
    #print(row)
        countryList=  df.ix[i]['solr_additions']
        if type(countryList) is str:
            countries = ""
            res = re.findall(r'\w+', countryList)
            for j in range(1,len(res)):
                countries += " " + res[j]
            df.at[i,'country'] =  countries.lower()
        else:
            df.at[i,'country'] =  ''
    return df

In [5]:
# The main fields of the metadata that will be used for the processing
#'notes' is a description on the dataset.
def project_dataframe(df):
    columns = ['title','notes','tags','organization','dataset_source','geodata', 'country']
    df = df[columns]
    return df

In [6]:
# extract the header of the dataset (the columns)
def extract_dataset_header(df):
    
    for i, row in df.iterrows():
        try:
            if df.ix[i]['id'] is not None and df.ix[i]['id'] != 0:
                columns= extract_resource_header(df.ix[i]['id'])
                df.at[i,'header'] = columns.lower()
            else:
                df.at[i,'header'] = ""
        except:
            #print("IO exception")
            continue
    return df

In [7]:
# the information of the organization that collect/share the dataset
def extract_org_info(df):
    
    for i, row in df.iterrows():
        tags = " "
        organization = df.ix[i]['organization']
        if type(organization) is str:
            organization = ast.literal_eval(organization)
        org = organization['description'] +" " + organization['title']
        df.at[i,'organization'] = org.lower()
        return df

In [8]:
# concatenate the content of all metadata fields into raw text
def get_text(df):
    #df.tags.astype(str)+ " " +
    textdata =  df.title.fillna('').astype(str)+" " + df.header.fillna('').astype(str) +" " + df.organization.fillna('').astype(str) +" " +  df.notes.fillna('').astype(str)+" "+ df.country.fillna('').astype(str) + " " + df.geodata.fillna('').astype(str) 
    return textdata

In [9]:
# concatenate the content of all metadata fields into raw text
def get_all_text(df):
    #df.tags.astype(str)+ " " +
    textdata =  df.title.fillna('').astype(str)+" " + df.header.fillna('').astype(str) +" " + df.organization.fillna('').astype(str) +" " +  df.notes.fillna('').astype(str)+" "+ df.country.fillna('').astype(str) + " " + df.geodata.fillna('').astype(str) 
    return textdata

In [10]:
## helper func.
from gensim.utils import simple_preprocess
def read_datasets(row):
    #print(row)
    return simple_preprocess(str(row).encode('utf-8'))

In [11]:
# helper func.
def print_word_table(table, key):
    return pd.DataFrame(table, columns=[key, 'similarity'])

In [12]:
# extract the current tags of the dataset mainly for comparison and evaluation 
def extract_tags(df):
    df['tag_list'] = [[] for _ in df.index]
    for i, row in df.iterrows():
        tags = ""
        tag_list = []
        tagvalues = df.ix[i]['tags']
        if type(tagvalues) is str:
            tagvalues = ast.literal_eval(tagvalues)
        if type(tagvalues) is float:
            tagvalues = []
        for t in tagvalues:
            #print( t['name'])
            if t['name'] is not None:
                tags+= t['name']+ " "
                tag_list.append(t['name'])
        if tags is None or tags.strip() =="":
            tags = " "
        df.at[i,'tags'] = tags.lower()
        df.at[i,'tag_list'] = tag_list
    return df

In [13]:
#def extract_resource_info(dataset):
 #   resource= dataset.get_resource()
  #  return resource['url'], resource['created'].split('-')[0]

In [14]:
# extract the resource objects for each dataset (if it has resource(s))
def extract_resource(datasets):
    dfObj = pd.DataFrame(columns=['id', 'package_id', 'url','date','format'])
    for ds in datasets:
        #print(ds)
        if ds.is_requestable() == False:
           #obj= None
            obj = ds.get_resource(index=0)
            #print(obj['format'])
            dfObj = dfObj.append({'id': obj['id'], 'package_id':obj['package_id'],'url': obj['url'], 'date':obj['created'].split('-')[0],'format':obj['format'] },ignore_index=True)
    return dfObj

In [15]:
# read the xls and/or csv of using the url of the resources associated with the dataset
def extract_resource_header(ds_id):
    #resource = Resource.read_from_hdx(str(ds_id))
    #if resource is not None:
    #    header = ""

    #    if resource['format'] == 'CSV':
            #print(resource['url'])
            #print("csv" ,resource['url'])
     #       s=requests.get(resource['url']).text
     #       c=pd.read_csv(StringIO(s), nrows=2)#, header=None, error_bad_lines=False)
     #       col_name = list(c.columns.values)
     #       header = ' '.join(str(e) for e in col_name)
     #   elif resource['format'] == 'XLS' or resource['format'] == 'XLSX':
     #       c=pd.read_excel(resource['url'])#, sheetname=0, header=1)#StringIO(s)) #read_excel(BytesIO(s))#,header=None) #read_excel(StringIO(s))
     #       col_name = list(c.columns.values)
     #       header = ' '.join(str(e) for e in col_name)
     #   else:
     #       header = ""
       # except IOError:
         #   print("IO exception")
          #  continue
            #pass
            
    #else:
        #header = ""
    header = ""
    return header
    #return resource['url'], resource['created'].split('-')[0]

In [16]:
## Connecting to HDX API
Configuration.create(hdx_site='prod', user_agent='Crawling - Education datasets', hdx_read_only=True)

'https://data.humdata.org/'

In [17]:
# test
extract_resource_header('cf0e8c85-e365-40f6-a7ab-58e98ad31e46')

''

In [18]:
#cleaned_dataset=pd.read_csv("after_clean.csv")

In [19]:
'''
This code snippets:
 1) Access HDX API and request all the datasets that have tag = 'education'.
 2) create a dataframe from the json of datasets
 3) extract resource objects for each dataset (if the dataset has any).
 4) use 'id', 'package_id' to join the resource object with the the dataset object.
 5) extract the header (column names) of the datsets if it has url (public url)
 6) save content to a file for further processing
'''
print("Getting Datasets")
datasets = Dataset.search_in_hdx(fq='', rows=100000)
print('creating data frame')
df = pd.DataFrame.from_dict(datasets, orient='columns')
print('resource')
df_resource= extract_resource(datasets)
print('joining data')
df.rename(columns={'id':'package_id'}, inplace=True)
df = pd.merge(df, df_resource, on='package_id', how='outer')
print('filling gaps')
df['id'].fillna(0, inplace=True)
df = extract_dataset_header(df)
df.to_csv("metadat_before_cleaning_new.csv", index=False)


## Or load from prevoiusly prepared file 
#df = pd.read_csv("metadat_before_cleaning_new.csv")

    

Getting Datasets
creating data frame
resource
joining data
filling gaps


In [20]:
'''
This code snippets:
 1) add geodata
 2) add country
 3) extract org_info
 4) extract tags
 5) clean the content (number, special character,links and/or non-english chars removal )
 6) concatenate the text content from all metdata fields.
'''

columns = ['id','header','title','tags','notes','total_res_downloads','organization','dataset_source','geodata', 'country']#,'location']
Pcolumns = ['title','header','tags','notes','organization','dataset_source','geodata', 'country']#,'location']
df = add_geodata(df)
df = add_country(df)
df_process = df[['title','notes','tags','header','organization','dataset_source','geodata', 'country']] #'tags'
df_process = extract_org_info(df_process)
df_process=  extract_tags(df_process)
df_process = Utility.data_clean(df_process,Pcolumns)
df_process['doc']  =  get_text(df_process)
df_process['All_text'] = get_all_text(df_process)
#text_data = df_process['doc'] #get_text(df_process)

In [21]:
df_process.ix[0]['doc']

'c te ivoire food security indicators  food agriculture organization united nations food agriculture organization statistics collates disseminates food agricultural statistics globally division develops methodologies standards data collection holds regular meetings workshops support member countries develop statistical systems produce publications working papers statistical yearbooks that cover food security prices production trade agri environmental statistics u f te ivoire '

In [22]:
# the tag_list of each dataset.
df_process['tag_list']

0                                     [food security, hxl]
1                                     [food security, hxl]
2                                     [food security, hxl]
3                                     [food security, hxl]
4                                     [food security, hxl]
                               ...                        
10081    [airports, buildings, earthquakes, education, ...
10082                   [cyclones - hurricanes - typhoons]
10083    [cyclones - hurricanes - typhoons, tropical cy...
10084    [cyclones - hurricanes - typhoons, geodata, ty...
10085    [ebola, geodata, health, logistics, water sani...
Name: tag_list, Length: 10086, dtype: object

In [23]:
stop_words= set(stopwords.words('english'))
stop_words.update(['unnamed','nan','file','xls','xlsx','zip','link', 'description','https'])



In [24]:
# save content after cleaning
#df_process.to_csv("after_clean.csv", index=False)

In [25]:
df_process.to_csv("after_clean_new.csv", index=False)

In [26]:
## stop words removal.
df_process['doc'] = Utility.remove_stopwords(df_process['doc'], stop_words)

In [27]:
#text_data =remove_stopwords(df_process['doc'])

In [28]:
#all_text_data =remove_stopwords(df_process['All_text'])

In [29]:
#df_process['All_text'] =remove_stopwords(df_process['All_text'])

In [30]:
## save preprocessed content into file
df_process.to_csv("after_preprocessing_new.csv", index=False)


In [31]:
df_process.ix[0]['doc']

'c te ivoire food security indicators  food agriculture organization united nations food agriculture organization statistics collates disseminates food agricultural statistics globally division develops methodologies standards data collection holds regular meetings workshops support member countries develop statistical systems produce publications working papers statistical yearbooks cover food security prices production trade agri environmental statistics u f te ivoire'